# PAN 2025 - AI-Generated Text Classifier with RoBERTa

This notebook implements a text classifier to detect AI-generated content using the RoBERTa model. The project includes data preprocessing, class balancing, data augmentation, and model training.

## 1. Library Imports

In [29]:
# Main libraries for data manipulation and machine learning
import pandas as pd
import numpy as np
import random
from collections import Counter

# Sklearn libraries for balancing and metrics
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

# Transformers and PyTorch libraries
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    EvalPrediction
)

# Utilities
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully")

All libraries imported successfully


## 2. Dataset Loading and Exploration

In [30]:
# Load the training dataset
df = pd.read_csv(r'data/dat_train_v2.csv')

# Explore basic dataset characteristics
print("=" * 50)
print("DATASET EXPLORATION")
print("=" * 50)
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

print("\n" + "=" * 30)
print("FIRST 5 ROWS:")
print("=" * 30)
print(df.head())

print("\n" + "=" * 30)
print("CLASS DISTRIBUTION:")
print("=" * 30)
class_distribution = df['label'].value_counts().sort_index()
print(class_distribution)

# Calculate percentages
total_samples = len(df)
print("\nPercentages by class:")
for label, count in class_distribution.items():
    percentage = (count / total_samples) * 100
    print(f"  Class {label}: {count:,} samples ({percentage:.2f}%)")

DATASET EXPLORATION
Dataset shape: (288918, 2)
Columns: ['text', 'label']

FIRST 5 ROWS:
                                                text  label
0  Have you ever had to wait for something for a ...      4
1  But now, things were not so simple._SEP_The gi...      3
2  Dear Editor,  I am writing to express my opini...      4
3  Humans once wielded formidable magical power. ...      4
4  Here is a way that I had to be patient, and we...      4

CLASS DISTRIBUTION:
label
0    75270
1    95398
2    91232
3    10740
4    14910
5     1368
Name: count, dtype: int64

Percentages by class:
  Class 0: 75,270 samples (26.05%)
  Class 1: 95,398 samples (33.02%)
  Class 2: 91,232 samples (31.58%)
  Class 3: 10,740 samples (3.72%)
  Class 4: 14,910 samples (5.16%)
  Class 5: 1,368 samples (0.47%)


## 3. Tokenizer Initialization and Verification

In [31]:
# Initialize RoBERTa tokenizer
print("Initializing RoBERTa tokenizer...")
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Verify tokenizer functionality with an example
sample_text = df['text'].iloc[0]
tokens = tokenizer(sample_text, truncation=True, max_length=512)

print("Tokenizer initialized successfully")
print(f"Example text: {sample_text[:100]}...")
print(f"Number of tokens: {len(tokens['input_ids'])}")
print(f"Tokenizer vocabulary: {tokenizer.vocab_size:,} tokens")

Initializing RoBERTa tokenizer...
Tokenizer initialized successfully
Example text: Have you ever had to wait for something for a really long time? Like waiting for your favorite food ...
Number of tokens: 230
Tokenizer vocabulary: 50,265 tokens


## 4. Class Distribution Analysis and Balancing Strategy

In [32]:
def analyze_class_distribution(df):
    """
    Analyzes class distribution and defines a balancing strategy.

    Args:
        df (pd.DataFrame): Dataset with 'text' and 'label' columns

    Returns:
        dict: Dictionary with target sizes for each class
    """
    print("CLASS DISTRIBUTION ANALYSIS")
    print("=" * 50)

    # Separate dataset by classes
    class_dfs = {}
    for label in sorted(df['label'].unique()):
        class_dfs[label] = df[df['label'] == label]
        print(f"  Class {label}: {len(class_dfs[label]):,} samples")

    # Define balancing strategy
    target_sizes = {}
    print("\nBALANCING STRATEGY:")
    print("-" * 30)

    for label, class_df in class_dfs.items():
        current_size = len(class_df)

        if current_size > 80000:
            target_sizes[label] = 80000  # Undersample majority classes
            action = "UNDERSAMPLE"
        elif current_size < 10000:
            target_sizes[label] = 10000  # Oversample minority classes
            action = "OVERSAMPLE"
        else:
            target_sizes[label] = current_size  # Keep balanced classes
            action = "MAINTAIN"

        print(f"  Class {label}: {current_size:,} → {target_sizes[label]:,} ({action})")

    return class_dfs, target_sizes

# Execute analysis
class_dfs, target_sizes = analyze_class_distribution(df)

CLASS DISTRIBUTION ANALYSIS
  Class 0: 75,270 samples
  Class 1: 95,398 samples
  Class 2: 91,232 samples
  Class 3: 10,740 samples
  Class 4: 14,910 samples
  Class 5: 1,368 samples

BALANCING STRATEGY:
------------------------------
  Class 0: 75,270 → 75,270 (MAINTAIN)
  Class 1: 95,398 → 80,000 (UNDERSAMPLE)
  Class 2: 91,232 → 80,000 (UNDERSAMPLE)
  Class 3: 10,740 → 10,740 (MAINTAIN)
  Class 4: 14,910 → 14,910 (MAINTAIN)
  Class 5: 1,368 → 10,000 (OVERSAMPLE)


## 5. Applying Balancing Techniques

In [33]:
def apply_resampling(class_dfs, target_sizes, random_state=42):
    """
    Applies resampling techniques to balance classes.

    Args:
        class_dfs (dict): Dictionary with DataFrames by class
        target_sizes (dict): Target sizes for each class
        random_state (int): Seed for reproducibility

    Returns:
        pd.DataFrame: Balanced dataset
    """
    print("APPLYING BALANCING TECHNIQUES")
    print("=" * 40)

    resampled_dfs = []

    for label, class_df in class_dfs.items():
        target_size = target_sizes[label]
        current_size = len(class_df)

        if current_size > target_size:
            # Undersample (reduce samples)
            resampled_df = resample(
                class_df,
                n_samples=target_size,
                replace=False,  # Without replacement
                random_state=random_state
            )
            print(f"Class {label}: Undersampled from {current_size:,} to {target_size:,}")

        elif current_size < target_size:
            # Oversample (increase samples)
            resampled_df = resample(
                class_df,
                n_samples=target_size,
                replace=True,   # With replacement
                random_state=random_state
            )
            print(f"Class {label}: Oversampled from {current_size:,} to {target_size:,}")

        else:
            # Keep the same
            resampled_df = class_df
            print(f"Class {label}: Maintained at {current_size:,}")

        resampled_dfs.append(resampled_df)

    # Combine all dataframes
    df_balanced = pd.concat(resampled_dfs, ignore_index=True)

    # Shuffle randomly
    df_balanced = df_balanced.sample(frac=1, random_state=random_state).reset_index(drop=True)

    print(f"\nBalanced dataset created: {len(df_balanced):,} total samples")
    return df_balanced

# Apply balancing
df_balanced = apply_resampling(class_dfs, target_sizes)

# Verify final distribution
print("\nFINAL DISTRIBUTION AFTER BALANCING:")
print(df_balanced['label'].value_counts().sort_index())

APPLYING BALANCING TECHNIQUES
Class 0: Maintained at 75,270
Class 1: Undersampled from 95,398 to 80,000
Class 2: Undersampled from 91,232 to 80,000
Class 3: Maintained at 10,740
Class 4: Maintained at 14,910
Class 5: Oversampled from 1,368 to 10,000

Balanced dataset created: 270,920 total samples

FINAL DISTRIBUTION AFTER BALANCING:
label
0    75270
1    80000
2    80000
3    10740
4    14910
5    10000
Name: count, dtype: int64


## 6. Data Augmentation for Minority Classes

In [34]:
def augment_text(text, num_augmentations=1):
    """
    Applies text augmentation techniques to increase diversity.

    Implemented techniques:
    - Random word deletion
    - Word position swapping
    - Word duplication

    Args:
        text (str): Original text
        num_augmentations (int): Number of augmentations to generate

    Returns:
        str: Augmented text
    """
    words = text.split()

    if len(words) < 5:  # Very short texts are not augmented
        return text

    # Randomly select augmentation technique
    aug_type = random.choice(['delete', 'swap', 'duplicate'])
    aug_words = words.copy()

    if aug_type == 'delete' and len(words) > 10:
        # Randomly delete 10% of words
        num_delete = max(1, int(len(words) * 0.1))
        for _ in range(num_delete):
            if len(aug_words) > 5:
                idx = random.randint(0, len(aug_words)-1)
                aug_words.pop(idx)

    elif aug_type == 'swap' and len(words) > 2:
        # Swap random words
        num_swaps = max(1, int(len(words) * 0.1))
        for _ in range(num_swaps):
            idx1 = random.randint(0, len(aug_words)-1)
            idx2 = random.randint(0, len(aug_words)-1)
            aug_words[idx1], aug_words[idx2] = aug_words[idx2], aug_words[idx1]

    elif aug_type == 'duplicate':
        # Duplicate random words
        if len(aug_words) > 0:
            idx = random.randint(0, len(aug_words)-1)
            aug_words.insert(idx, aug_words[idx])

    return ' '.join(aug_words)

def apply_data_augmentation(df_balanced, class_dfs, augmentation_prob=0.5, random_state=42):
    """
    Applies data augmentation to classes that were oversampled.

    Args:
        df_balanced (pd.DataFrame): Already balanced dataset
        class_dfs (dict): Original DataFrames by class
        augmentation_prob (float): Probability of augmenting a text
        random_state (int): Seed for reproducibility

    Returns:
        pd.DataFrame: Dataset with augmented data
    """
    print("APPLYING DATA AUGMENTATION")
    print("=" * 35)

    # Set seed
    random.seed(random_state)

    # Identify minority classes (that were oversampled)
    minority_classes = [label for label, df in class_dfs.items() if len(df) < 10000]
    print(f"Classes to augment: {minority_classes}")

    augmented_rows = []

    # Apply augmentation with progress bar
    for idx, row in tqdm(df_balanced.iterrows(), total=len(df_balanced), desc="Augmenting data"):
        if row['label'] in minority_classes and random.random() < augmentation_prob:
            augmented_text = augment_text(row['text'])
            augmented_rows.append({'text': augmented_text, 'label': row['label']})

    # Add augmented rows to dataset
    if augmented_rows:
        df_augmented = pd.DataFrame(augmented_rows)
        df_balanced = pd.concat([df_balanced, df_augmented], ignore_index=True)
        print(f"Added {len(augmented_rows):,} augmented texts")

        # Shuffle again
        df_balanced = df_balanced.sample(frac=1, random_state=random_state).reset_index(drop=True)
    else:
        print("No augmented texts were added")

    return df_balanced

# Apply augmentation
df_balanced = apply_data_augmentation(df_balanced, class_dfs)

# Show final distribution
print("\nFINAL DISTRIBUTION WITH AUGMENTATION:")
final_distribution = df_balanced['label'].value_counts().sort_index()
print(final_distribution)

APPLYING DATA AUGMENTATION
Classes to augment: [np.int64(5)]


Augmenting data: 100%|██████████| 270920/270920 [00:05<00:00, 47813.70it/s]

Added 5,048 augmented texts

FINAL DISTRIBUTION WITH AUGMENTATION:
label
0    75270
1    80000
2    80000
3    10740
4    14910
5    15048
Name: count, dtype: int64


## 7. Saving Processed Dataset

In [35]:
# Save the processed dataset
output_filename = 'dataset_balanced_roberta.csv'
df_balanced.to_csv(output_filename, index=False)

print("DATASET SAVED SUCCESSFULLY")
print("=" * 35)
print(f"Filename: {output_filename}")
print(f"Total samples: {len(df_balanced):,}")
print(f"Final distribution:")
for label, count in df_balanced['label'].value_counts().sort_index().items():
    print(f"   Class {label}: {count:,} samples")

# Show a sample of the final dataset
print(f"\nProcessed dataset sample:")
print(df_balanced.head(3))

DATASET SAVED SUCCESSFULLY
Filename: dataset_balanced_roberta.csv
Total samples: 275,968
Final distribution:
   Class 0: 75,270 samples
   Class 1: 80,000 samples
   Class 2: 80,000 samples
   Class 3: 10,740 samples
   Class 4: 14,910 samples
   Class 5: 15,048 samples

Processed dataset sample:
                                                text  label
0  The dangers of using a cellphone while driving...      2
1  To understand this, it's helpful to start with...      1
2  The concept of driverless cars may appear fun,...      1


## 8. Data Preparation for Training

In [37]:
def prepare_train_val_test_splits(df, test_size=0.2, val_size=0.5, random_state=42):
    """
    Splits the dataset into training, validation, and test sets.

    Args:
        df (pd.DataFrame): Complete dataset
        test_size (float): Proportion for test+validation
        val_size (float): Proportion of validation within test+val set
        random_state (int): Seed for reproducibility

    Returns:
        tuple: (train_texts, val_texts, test_texts, train_labels, val_labels, test_labels)
    """
    print("PREPARING DATA SPLITS")
    print("=" * 35)

    # First split: train vs (val + test)
    train_texts, temp_texts, train_labels, temp_labels = train_test_split(
        df['text'].tolist(),
        df['label'].tolist(),
        test_size=test_size,
        random_state=random_state,
        stratify=df['label']
    )

    # Second split: val vs test
    val_texts, test_texts, val_labels, test_labels = train_test_split(
        temp_texts,
        temp_labels,
        test_size=val_size,
        random_state=random_state,
        stratify=temp_labels
    )

    print(f"Training: {len(train_texts):,} samples ({len(train_texts)/len(df)*100:.1f}%)")
    print(f"Validation: {len(val_texts):,} samples ({len(val_texts)/len(df)*100:.1f}%)")
    print(f"Test: {len(test_texts):,} samples ({len(test_texts)/len(df)*100:.1f}%)")

    return train_texts, val_texts, test_texts, train_labels, val_labels, test_labels

# Reload the processed dataset
df = pd.read_csv('dataset_balanced_roberta.csv')

# Prepare splits
train_texts, val_texts, test_texts, train_labels, val_labels, test_labels = prepare_train_val_test_splits(df)

# Verify stratified distribution in each set
print("\nStratified distribution verification:")
for split_name, labels in [("Train", train_labels), ("Val", val_labels), ("Test", test_labels)]:
    distribution = pd.Series(labels).value_counts().sort_index()
    print(f"\n{split_name}:")
    for label, count in distribution.items():
        print(f"  Class {label}: {count:,}")

PREPARING DATA SPLITS
Training: 220,774 samples (80.0%)
Validation: 27,597 samples (10.0%)
Test: 27,597 samples (10.0%)

Stratified distribution verification:

Train:
  Class 0: 60,216
  Class 1: 64,000
  Class 2: 64,000
  Class 3: 8,592
  Class 4: 11,928
  Class 5: 12,038

Val:
  Class 0: 7,527
  Class 1: 8,000
  Class 2: 8,000
  Class 3: 1,074
  Class 4: 1,491
  Class 5: 1,505

Test:
  Class 0: 7,527
  Class 1: 8,000
  Class 2: 8,000
  Class 3: 1,074
  Class 4: 1,491
  Class 5: 1,505


## 9. Class Weights Calculation

In [38]:
def calculate_class_weights(df):
    """
    Calculates class weights to handle residual imbalance.

    Args:
        df (pd.DataFrame): Dataset with class distribution

    Returns:
        tuple: (weights_dict, weights_tensor)
    """
    print("CALCULATING CLASS WEIGHTS")
    print("=" * 30)

    # Get unique labels sorted
    unique_labels = sorted(df['label'].unique())
    unique_labels_array = np.array(unique_labels)

    # Calculate balanced weights
    class_weights = compute_class_weight(
        'balanced',
        classes=unique_labels_array,
        y=df['label'].values
    )

    # Create weights dictionary
    weights_dict = {label: weight for label, weight in zip(unique_labels, class_weights)}

    print("Calculated weights:")
    for label, weight in weights_dict.items():
        count = df[df['label'] == label].shape[0]
        print(f"  Class {label}: weight = {weight:.4f}, samples = {count:,}")

    # Convert to PyTorch tensor
    weights_tensor = torch.tensor(class_weights, dtype=torch.float)
    print(f"\nWeights tensor: {weights_tensor}")

    return weights_dict, weights_tensor

# Calculate weights
weights_dict, weights_tensor = calculate_class_weights(df)

CALCULATING CLASS WEIGHTS
Calculated weights:
  Class 0: weight = 0.6111, samples = 75,270
  Class 1: weight = 0.5749, samples = 80,000
  Class 2: weight = 0.5749, samples = 80,000
  Class 3: weight = 4.2826, samples = 10,740
  Class 4: weight = 3.0848, samples = 14,910
  Class 5: weight = 3.0565, samples = 15,048

Weights tensor: tensor([0.6111, 0.5749, 0.5749, 4.2826, 3.0848, 3.0565])


## 10. Custom Dataset Class

In [39]:
class TextClassificationDataset(Dataset):
    """
    Custom dataset for text classification with RoBERTa.

    Features:
    - Automatic tokenization with truncation and padding
    - Label to index mapping
    - Compatible with PyTorch DataLoader
    """

    def __init__(self, texts, labels, tokenizer, max_length=512):
        """
        Initializes the dataset.

        Args:
            texts (list): List of texts
            labels (list): List of labels
            tokenizer: Transformers tokenizer
            max_length (int): Maximum sequence length
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Create label to index mapping
        self.unique_labels = sorted(list(set(labels)))
        self.label_to_id = {label: i for i, label in enumerate(self.unique_labels)}
        self.id_to_label = {i: label for label, i in self.label_to_id.items()}

        print(f"Dataset created: {len(self.texts)} samples")
        print(f"Label mapping: {self.label_to_id}")

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Gets a tokenized element from the dataset.

        Args:
            idx (int): Element index

        Returns:
            dict: Dictionary with input_ids, attention_mask, and labels
        """
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize text
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.label_to_id[label], dtype=torch.long)
        }

# Create datasets
print("CREATING CUSTOM DATASETS")
print("=" * 35)

train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer)
test_dataset = TextClassificationDataset(test_texts, test_labels, tokenizer)

print(f"Datasets created successfully")

CREATING CUSTOM DATASETS
Dataset created: 220774 samples
Label mapping: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}
Dataset created: 27597 samples
Label mapping: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}
Dataset created: 27597 samples
Label mapping: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}
Datasets created successfully


## 11. RoBERTa Model Configuration

In [40]:
def setup_roberta_model(num_labels, model_name='roberta-base'):
    """
    Configures RoBERTa model for sequence classification.

    Args:
        num_labels (int): Number of classes
        model_name (str): Pre-trained model name

    Returns:
        tuple: (model, device)
    """
    print("CONFIGURING ROBERTA MODEL")
    print("=" * 32)

    # Load pre-trained model
    model = RobertaForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
        problem_type="single_label_classification"
    )

    # Configure device (GPU if available)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Model information
    num_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Device: {device}")
    print(f"Total parameters: {num_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Number of classes: {num_labels}")

    return model, device

# Configure model
unique_labels = sorted(df['label'].unique())
model, device = setup_roberta_model(len(unique_labels))

# Move weights tensor to device
weights_tensor = weights_tensor.to(device)

CONFIGURING ROBERTA MODEL


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Device: cuda
Total parameters: 124,650,246
Trainable parameters: 124,650,246
Number of classes: 6


## 12. Custom Trainer with Class Weights

In [41]:
class WeightedTrainer(Trainer):
    """
    Custom trainer that includes class weights in the loss function.

    This helps handle class imbalance by applying higher penalty
    to incorrect predictions of minority classes.
    """

    def __init__(self, class_weights, *args, **kwargs):
        """
        Initializes the trainer with class weights.

        Args:
            class_weights (torch.Tensor): Tensor with weights for each class
        """
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights
        print(f"Trainer configured with class weights: {class_weights}")

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Computes loss using class weights.

        Args:
            model: Classification model
            inputs: Input batch
            return_outputs: Whether to return model outputs
            num_items_in_batch: Number of items in batch

        Returns:
            torch.Tensor or tuple: Loss (and outputs if return_outputs=True)
        """
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')

        # Apply class weights in loss function
        loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_pred: EvalPrediction):
    """
    Computes evaluation metrics for the model.

    Args:
        eval_pred: Predictions and true labels

    Returns:
        dict: Dictionary with calculated metrics
    """
    # Extract predictions
    predictions = eval_pred.predictions[0] if isinstance(eval_pred.predictions, tuple) else eval_pred.predictions
    predictions = np.argmax(predictions, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(eval_pred.label_ids, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        eval_pred.label_ids,
        predictions,
        average='weighted'
    )

    # F1 per individual class
    _, _, f1_per_class, _ = precision_recall_fscore_support(
        eval_pred.label_ids,
        predictions,
        average=None
    )

    return {
        'accuracy': accuracy,
        'f1_weighted': f1,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'f1_per_class': f1_per_class.tolist()
    }

print("Training functions defined correctly")

Training functions defined correctly


## 13. Training Arguments Configuration

In [42]:
def setup_training_arguments(output_dir='./roberta-classifier', epochs=3, batch_size=64):
    """
    Configures arguments for model training.

    Args:
        output_dir (str): Directory to save the model
        epochs (int): Number of training epochs
        batch_size (int): Batch size

    Returns:
        TrainingArguments: Training configuration
    """
    training_args = TrainingArguments(
        # Directories
        output_dir=output_dir,
        logging_dir='./logs',

        # Training configuration
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size + 32,  # Larger batch for evaluation

        # Optimization
        warmup_steps=500,
        weight_decay=0.01,
        learning_rate=2e-5,

        # Logging and evaluation
        logging_steps=500,
        eval_strategy="steps",
        eval_steps=1000,

        # Model saving
        save_strategy="steps",
        save_steps=1000,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model='f1_weighted',
        greater_is_better=True,

        # Performance optimizations
        fp16=True if torch.cuda.is_available() else False,
        gradient_checkpointing=True,
        dataloader_pin_memory=True,

        # Avoid external logging
        report_to="none"
    )

    print("TRAINING CONFIGURATION")
    print("=" * 33)
    print(f"Output directory: {output_dir}")
    print(f"Epochs: {epochs}")
    print(f"Batch size (train): {batch_size}")
    print(f"Batch size (eval): {batch_size + 32}")
    print(f"FP16: {training_args.fp16}")
    print(f"Gradient checkpointing: {training_args.gradient_checkpointing}")

    return training_args

# Configure training arguments
training_args = setup_training_arguments()

TRAINING CONFIGURATION
Output directory: ./roberta-classifier
Epochs: 3
Batch size (train): 64
Batch size (eval): 96
FP16: True
Gradient checkpointing: True


## 14. Trainer Creation and Configuration

In [43]:
# Create trainer with class weights
print("CONFIGURING TRAINER")
print("=" * 25)

trainer = WeightedTrainer(
    class_weights=weights_tensor,
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("Trainer configured successfully")
print(f"Model: {model.__class__.__name__}")
print(f"Training dataset: {len(train_dataset)} samples")
print(f"Validation dataset: {len(val_dataset)} samples")
print(f"Class weights applied: {len(weights_tensor)} classes")

CONFIGURING TRAINER
Trainer configured with class weights: tensor([0.6111, 0.5749, 0.5749, 4.2826, 3.0848, 3.0565], device='cuda:0')
Trainer configured successfully
Model: RobertaForSequenceClassification
Training dataset: 220774 samples
Validation dataset: 27597 samples
Class weights applied: 6 classes


## 15. Model Training

In [44]:
def train_model(trainer, save_path='./roberta-classifier-final'):
    """
    Trains the model and saves results.

    Args:
        trainer: Configured trainer
        save_path (str): Path to save final model

    Returns:
        TrainOutput: Training results
    """
    print("STARTING TRAINING")
    print("=" * 26)
    print("This process may take several hours...")

    try:
        # Train the model
        train_result = trainer.train()

        # Save final model
        trainer.save_model(save_path)
        print(f"Model saved at: {save_path}")

        # Show training metrics
        print("\nTRAINING RESULTS")
        print("=" * 31)
        print(f"Final loss: {train_result.training_loss:.4f}")
        print(f"Total steps: {train_result.global_step}")
        print(f"Total time: {train_result.metrics.get('train_runtime', 'N/A')}")

        return train_result

    except KeyboardInterrupt:
        print("\nTraining interrupted by user")
        print("Saving current model state...")
        trainer.save_model(save_path + "_interrupted")
        return None

    except Exception as e:
        print(f"\nError during training: {str(e)}")
        return None

# Execute training
train_result = train_model(trainer)

STARTING TRAINING
This process may take several hours...


Step,Training Loss,Validation Loss,Accuracy,F1 Weighted,Precision Weighted,Recall Weighted,F1 Per Class
1000,0.243400,0.238421,0.914991,0.915206,0.923297,0.914991,"[0.9728190876431912, 0.8929073739200322, 0.9136189481017067, 0.8101167315175097, 0.7648970747562297, 0.9779286926994907]"



Training interrupted by user
Saving current model state...


## 16. Model Evaluation

In [ ]:
def evaluate_model(trainer, test_dataset, id_to_label):
    """
    Evaluates the trained model on the test set.

    Args:
        trainer: Trained trainer
        test_dataset: Test dataset
        id_to_label: Index to label mapping

    Returns:
        dict: Evaluation metrics
    """
    print("EVALUATING MODEL ON TEST SET")
    print("=" * 41)

    # Evaluate on test set
    eval_results = trainer.evaluate(test_dataset)

    # Get detailed predictions
    predictions = trainer.predict(test_dataset)
    y_pred = np.argmax(predictions.predictions, axis=1)
    y_true = predictions.label_ids

    # Calculate per-class metrics
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average=None
    )

    print("GENERAL METRICS:")
    print(f"  Accuracy: {eval_results['eval_accuracy']:.4f}")
    print(f"  F1-Score (weighted): {eval_results['eval_f1_weighted']:.4f}")
    print(f"  Precision (weighted): {eval_results['eval_precision_weighted']:.4f}")
    print(f"  Recall (weighted): {eval_results['eval_recall_weighted']:.4f}")

    print("\nPER-CLASS METRICS:")
    for i, (p, r, f, s) in enumerate(zip(precision, recall, f1, support)):
        class_label = id_to_label[i]
        print(f"  Class {class_label}: Precision={p:.3f}, Recall={r:.3f}, F1={f:.3f}, Support={s}")

    # Confusion matrix
    conf_matrix = confusion_matrix(y_true, y_pred)
    print(f"\nCONFUSION MATRIX:")
    print(conf_matrix)

    return eval_results

# Evaluate model if training was successful
if train_result is not None:
    eval_results = evaluate_model(trainer, test_dataset, train_dataset.id_to_label)
else:
    print("Cannot evaluate: training not completed")

## 17. Final Model Saving

In [ ]:
def save_final_model(model, tokenizer, save_path="./RoBERTa_IA_Final"):
    """
    Saves the final model and tokenizer.

    Args:
        model: Trained model
        tokenizer: Used tokenizer
        save_path (str): Save path
    """
    print("SAVING FINAL MODEL")
    print("=" * 24)

    try:
        # Save model and tokenizer
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)

        print(f"Model saved successfully at: {save_path}")
        print(f"Saved files:")
        print(f"   - config.json")
        print(f"   - pytorch_model.bin")
        print(f"   - tokenizer_config.json")
        print(f"   - vocab.json")
        print(f"   - merges.txt")

        # Save additional information
        model_info = {
            'model_name': 'roberta-base',
            'num_labels': len(train_dataset.unique_labels),
            'label_mapping': train_dataset.label_to_id,
            'training_samples': len(train_dataset),
            'validation_samples': len(val_dataset),
            'test_samples': len(test_dataset)
        }

        import json
        with open(f"{save_path}/model_info.json", 'w') as f:
            json.dump(model_info, f, indent=2)

        print(f"   - model_info.json")
        print("Save completed!")

    except Exception as e:
        print(f"Error saving: {str(e)}")

# Save final model
save_final_model(model, tokenizer)

print("\nPROCESS COMPLETED SUCCESSFULLY!")
print("=" * 35)
print("Project summary:")
print(f"   - Original dataset: {len(df)} samples")
print(f"   - Balanced dataset: {len(df_balanced)} samples")
print(f"   - Classes: {len(train_dataset.unique_labels)}")
print(f"   - Model: RoBERTa for AI text classification")